# ModelChain

## A simple ModelChain example

Before delving into the intricacies of ModelChain, we provide a brief example of the modeling steps using ModelChain. First, we import pvlib's objects, module data, and inverter data.

In [1]:
import pandas as pd
import numpy as np

# pvlib imports
import pvlib

from pvlib.pvsystem import PVSystem
from pvlib.location import Location
from pvlib.modelchain import ModelChain

# load some module and inverter specifications
sandia_modules = pvlib.pvsystem.retrieve_sam('SandiaMod')
cec_inverters = pvlib.pvsystem.retrieve_sam('cecinverter')

sandia_module = sandia_modules['Canadian_Solar_CS5P_220M___2009_']
cec_inverter = cec_inverters['ABB__MICRO_0_25_I_OUTD_US_208_208V__CEC_2014_']

Now we create a Location object, a PVSystem object, and a ModelChain object.

In [2]:
location = Location(latitude=32.2, longitude=-110.9)
system = PVSystem(surface_tilt=20, surface_azimuth=200, 
                  module_parameters=sandia_module,
                  inverter_parameters=cec_inverter)
mc = ModelChain(system, location)

Printing a ModelChain object will display its models.

In [3]:
print(mc)

ModelChain: 
  name: None
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: sapm
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


Next, we run a model with some simple weather data.

In [4]:
weather = pd.DataFrame([[1050, 1000, 100, 30, 5]], 
                       columns=['ghi', 'dni', 'dhi', 'temp_air', 'wind_speed'], 
                       index=[pd.Timestamp('20170401 1200', tz='US/Arizona')])

mc.run_model(times=weather.index, weather=weather);

ModelChain stores the modeling results on a series of attributes. A few examples are shown below.

In [5]:
mc.aoi

2017-04-01 12:00:00-07:00    15.929176
Name: aoi, dtype: float64

In [6]:
mc.dc

,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx
2017-04-01 12:00:00-07:00,5.485958,4.860317,52.319047,40.585752,197.259628,5.363079,3.401315


In [7]:
mc.ac

2017-04-01 12:00:00-07:00    189.915445
dtype: float64

The remainder of this guide examines the ModelChain functionality and explores common pitfalls.

## Defining a ModelChain

Let's make the most basic Location and PVSystem objects and build from there.

In [ ]:
location = Location(32.2, -110.9)
poorly_specified_system = PVSystem()
print(location)
print(poorly_specified_system)

These basic objects do not have enough information for ModelChain to be able to automatically determine its set of models, so the ModelChain will throw an error when we try to create it.

In [ ]:
ModelChain(poorly_specified_system, location)

If our goal is simply to get the object constructed, we can specify the models that the ModelChain should use. We'll have to fill in missing data on the PVSystem object later, but maybe that's desirable in some workflows.

In [ ]:
mc = ModelChain(poorly_specified_system, location, 
                dc_model='singlediode', ac_model='snlinverter', 
                aoi_model='physical', spectral_model='no_loss')
print(mc)

In [ ]:
mc.run_model(times=weather.index, weather=weather)

Next, we define a PVSystem with a module from the SAPM database and an inverter from the CEC database. ModelChain will examine the PVSystem object's properties and determine that it should choose the SAPM DC model, AC model, AOI loss model, and spectral loss model.

In [ ]:
sapm_system = PVSystem(module_parameters=sandia_module, inverter_parameters=cec_inverter)
mc = ModelChain(system, location)
print(mc)

In [ ]:
mc.run_model(times=weather.index, weather=weather)
mc.ac

Alternatively, we could have specified single diode or PVWatts related information in the PVSystem construction. Here we pass PVWatts data to the PVSystem. ModelChain will automatically determine that it should choose PVWatts DC and AC models. ModelChain still needs us to specify ``aoi_model`` and ``spectral_model`` keyword arguments because the ``system.module_parameters`` dictionary does not contain enough information to determine which of those models to choose.

In [ ]:
pvwatts_system = PVSystem(module_parameters={'pdc0': 240, 'gamma_pdc': -0.004})
mc = ModelChain(pvwatts_system, location, 
                aoi_model='physical', spectral_model='no_loss')
print(mc)

In [ ]:
mc.run_model(times=weather.index, weather=weather)
mc.ac

User-supplied keyword arguments override ModelChain's inspection methods. For example, we can tell ModelChain to use different loss functions for a PVSystem that contains SAPM-specific parameters. 

In [ ]:
sapm_system = PVSystem(module_parameters=sandia_module, inverter_parameters=cec_inverter)
mc = ModelChain(system, location, aoi_model='physical', spectral_model='no_loss')
print(mc)

In [ ]:
mc.run_model(times=weather.index, weather=weather)
mc.ac

## Demystifying ModelChain internals

### run_model

In [ ]:
np.source(mc.run_model)

### Wrapping methods into a unified API

Readers may notice that the source code of the ModelChain.run_model method is model-agnostic. ModelChain.run_model calls generic methods such as ``self.dc_model`` rather than a specific model such as ``singlediode``. So how does the ModelChain.run_model know what models it's supposed to run? The answer comes in two parts, and allows us to explore more of the ModelChain API along the way.

First, ModelChain has a set of methods that wrap the PVSystem methods that perform the calculations (or further wrap the pvsystem.py module's functions). Each of these methods takes the same arguments (``self``) and sets the same attributes, thus creating a uniform API. For example, the ModelChain.pvwatts_dc method is shown below. Its only argument is ``self``, and it sets the ``dc`` attribute.

In [ ]:
np.source(mc.pvwatts_dc)

The ModelChain.pvwatts_dc method calls the pvwatts_dc method of the PVSystem object that we supplied using data that is stored in its own ``effective_irradiance`` and ``temps`` attributes. Then it assigns the result to the ``dc`` attribute of the ModelChain object. The code below shows a simple example of this.

In [ ]:
# make the objects
pvwatts_system = PVSystem(module_parameters={'pdc0': 240, 'gamma_pdc': -0.004})
mc = ModelChain(pvwatts_system, location, 
                aoi_model='no_loss', spectral_model='no_loss')

# manually assign data to the attributes that ModelChain.pvwatts_dc will need.
# for standard workflows, run_model would assign these attributes.
mc.effective_irradiance = pd.Series(1000, index=[pd.Timestamp('20170401 1200-0700')])
mc.temps = pd.DataFrame({'temp_cell': 50, 'temp_module': 50}, index=[pd.Timestamp('20170401 1200-0700')])

# run ModelChain.pvwatts_dc and look at the result
mc.pvwatts_dc()
mc.dc

The ModelChain.sapm method works similarly to the ModelChain.pvwatts_dc method. It calls the PVSystem.sapm method using stored data, then assigns the result to the ``dc`` attribute. The ModelChain.sapm method differs from the ModelChain.pvwatts_dc method in three notable ways. First, the PVSystem.sapm method expects different units for effective irradiance, so ModelChain handles the conversion for us. Second, the PVSystem.sapm method (and the PVSystem.singlediode method) returns a DataFrame with current, voltage, and power parameters rather than a simple Series of power. Finally, this current and voltage information allows the SAPM and single diode model paths to support the concept of modules in series and parallel, which is handled by the PVSystem.scale_voltage_current_power method. 

In [ ]:
np.source(mc.sapm)

In [ ]:
# make the objects
sapm_system = PVSystem(module_parameters=sandia_module, inverter_parameters=cec_inverter)
mc = ModelChain(sapm_system, location)

# manually assign data to the attributes that ModelChain.sapm will need.
# for standard workflows, run_model would assign these attributes.
mc.effective_irradiance = pd.Series(1000, index=[pd.Timestamp('20170401 1200-0700')])
mc.temps = pd.DataFrame({'temp_cell': 50, 'temp_module': 50}, index=[pd.Timestamp('20170401 1200-0700')])

# run ModelChain.sapm and look at the result
mc.sapm()
mc.dc

We've established that the ``ModelChain.pvwatts_dc`` and ``ModelChain.sapm`` have the same API: they take the same arugments (``self``) and they both set the ``dc`` attribute.\* Because the methods have the same API, we can call them in the same way. ModelChain includes a large number of methods that perform the same API-unification roles for each modeling step.

Again, so how does the ModelChain.run_model know which models it's supposed to run?

At object construction, ModelChain assigns the desired model's method (e.g. ``ModelChain.pvwatts_dc``) to the corresponding generic attribute (e.g. ``ModelChain.dc_model``) using a method described in the next section.

In [ ]:
pvwatts_system = PVSystem(module_parameters={'pdc0': 240, 'gamma_pdc': -0.004})
mc = ModelChain(pvwatts_system, location, 
                aoi_model='no_loss', spectral_model='no_loss')
mc.dc_model.__func__

The ModelChain.run_model method can ignorantly call ``self.dc_module`` because the API is the same for all methods that may be assigned to this attribute.

\* some readers may object that the API is *not* actually the same because the type of the ``dc`` attribute is different (Series vs. DataFrame)!

### Inferring models

How does ModelChain infer the appropriate model types? ModelChain uses a series of methods (ModelChain.infer_dc_model, ModelChain.infer_ac_model, etc.) that examine the user-supplied PVSystem object. The inference methods use set logic to assign one of the model-specific methods, such as ModelChain.sapm or ModelChain.snlinverter, to the universal method names ModelChain.dc_model and ModelChain.ac_model. A few examples are shown below.

In [ ]:
np.source(mc.infer_dc_model)

In [ ]:
np.source(mc.infer_ac_model)

## User-defined models

Users may also write their own functions and pass them as arguments to ModelChain. The first argument of the function must be a ModelChain instance. For example, the functions below implement the PVUSA model and a wrapper function appropriate for use with ModelChain. This follows the pattern of implementing the core models using the simplest possible functions, and then implementing wrappers to make them easier to use in specific applications. Of course, you could implement it in a single function if you wanted to.

In [ ]:
def pvusa(poa_global, wind_speed, temp_air, a, b, c, d):
    """
    Calculates system power according to the PVUSA equation
    
    P = I * (a + b*I + c*W + d*T)
    
    where
    P is the output power,
    I is the plane of array irradiance,
    W is the wind speed, and
    T is the temperature
    a, b, c, d are empirically derived parameters.
    """
    return poa_global * (a + b*poa_global + c*wind_speed + d*temp_air)


def pvusa_mc_wrapper(mc):
    # calculate the dc power and assign it to mc.dc
    mc.dc = pvusa(mc.total_irrad['poa_global'], mc.weather['wind_speed'], mc.weather['temp_air'],
                  mc.system.module_parameters['a'], mc.system.module_parameters['b'],
                  mc.system.module_parameters['c'], mc.system.module_parameters['d'])
    
    # returning mc is optional, but enables method chaining
    return mc


def pvusa_ac_mc_wrapper(mc):
    # keep it simple
    mc.ac = mc.dc
    return mc

In [ ]:
module_parameters = {'a': 0.2, 'b': 0.00001, 'c': 0.001, 'd': -0.00005}
pvusa_system = PVSystem(module_parameters=module_parameters)

mc = ModelChain(pvusa_system, location, 
                dc_model=pvusa_mc_wrapper, ac_model=pvusa_ac_mc_wrapper,
                aoi_model='no_loss', spectral_model='no_loss')

A ModelChain object uses Python's functools.partial function to assign itself as the argument to the user-supplied functions.

In [ ]:
mc.dc_model.func

The end result is that ModelChain.run_model works as expected!

In [ ]:
mc.run_model(times=weather.index, weather=weather)
mc.dc

## Campanelli et al. Model

The model of Campanelli et al. is available in the `pvsystem` module as a collection of lower-level functions that can be used in multiple ways with the `ModelChain` API.

### Use module parameters fit from data at pv-fit.com to define a PVSystem

In [8]:
import pvlib.pvsystem as pvsystem

# dict defined from JSON response subset from a pv-fit.com model calibration for a 72-cell, 275 W module at SRC
fit_params = {'n_mod_V_0': 1.8760873861375469, 'i_sc_A_0': 8.300642232880616,
              'alpha_bg_mod_0': -0.0794653116276011, 'i_rs_A_0': 3.1370554096867056e-10,
              'g_p_Mho_0': 0.0026062186459665454, 'bg_mod_eV_0': 80.76475208957835,
              'r_s_Ohm_0': 0.5297525695641349}

# Translate results into module_parameters expected by pvlib-python (TODO Rename to avoid this)
module_params_campanelli = {'Rs': fit_params['r_s_Ohm_0'],
                            'Gsh': fit_params['g_p_Mho_0'],
                            'nNsVth': pvsystem.get_nNsVth_campanelli,
                            'nNsVth_ref': fit_params['n_mod_V_0'],
                            'I0': pvsystem.get_I0_campanelli,
                            'I0_ref': fit_params['i_rs_A_0'],
                            'Eg_ref_star': fit_params['bg_mod_eV_0'],
                            'dEgdT_ref_star': fit_params['alpha_bg_mod_0'],
                            'IL': pvsystem.get_IL_campanelli,
                            'Isc_ref': fit_params['i_sc_A_0']}

### Model wrapper for use with matched reference device effective irradiance and cell temperature

In [9]:
# TODO Breifly explain how this model is calibrated with F and H and best used with matched ref devices

# Define the PVSystem and ModelChain
system_campanelli = PVSystem(surface_tilt=20, surface_azimuth=200,
                             module_parameters=module_params_campanelli,
                             inverter_parameters=cec_inverter)


### Model wrapper for use with SAPM effective irradiance and cell temperature

In [10]:
from pvlib.modelchain import sdm_campanelli_sapm

# To use SAPM, we need to include a short-circuit temperature coefficient and reference temp
# In practice, try to match your module as best you can to one from the SAPM database
module_params_campanelli_sapm = {**module_params_campanelli, **sandia_module.to_dict()}
module_params_campanelli_sapm['temp_ref'] = 25.  # degC
    
# Define the PVSystem and ModelChain
system_campanelli_sapm = PVSystem(surface_tilt=20, surface_azimuth=200,
                                  module_parameters=module_params_campanelli_sapm,
                                  inverter_parameters=cec_inverter)
mc_campanelli = ModelChain(system_campanelli_sapm, location, dc_model=sdm_campanelli_sapm, name="Campanelli")
print(mc_campanelli)
mc_campanelli.run_model(times=weather.index, weather=weather)

ModelChain: 
  name: Campanelli
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: functools.partial(<function sdm_campanelli_sapm at 0x7f266c8d51e0>, ModelChain: 
  name: Campanelli
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: ...
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses)
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses


/home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/pvsystem.py:1250: RuntimeWarning: overflow encountered in expm1
  kwargs['Rs'] / kwargs['nNsVth']) + \
/home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/pvsystem.py:2280: RuntimeWarning: invalid value encountered in subtract
  (Rs[idx_p]*Gsh[idx_p] + 1.) - (a[idx_p]/Rs[idx_p])*lambertwterm
/home/campy/Documents/IMS/SolarPV/PVfit/pvlib-python/pvlib/pvsystem.py:2154: RuntimeWarning: invalid value encountered in add
  w = w * (1. - np.log(w) + logargW) / (1. + w)


ModelChain: 
  name: Campanelli
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: functools.partial(<function sdm_campanelli_sapm at 0x7f266c8d51e0>, ModelChain: 
  name: Campanelli
  orientation_strategy: None
  clearsky_model: ineichen
  transposition_model: haydavies
  solar_position_method: nrel_numpy
  airmass_model: kastenyoung1989
  dc_model: ...
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses)
  ac_model: snlinverter
  aoi_model: sapm_aoi_loss
  spectral_model: sapm_spectral_loss
  temp_model: sapm_temp
  losses_model: no_extra_losses

In [11]:
mc.aoi

2017-04-01 12:00:00-07:00    15.929176
Name: aoi, dtype: float64

In [12]:
mc.dc

,i_sc,i_mp,v_oc,v_mp,p_mp,i_x,i_xx
2017-04-01 12:00:00-07:00,5.485958,4.860317,52.319047,40.585752,197.259628,5.363079,3.401315


In [13]:
mc.ac

2017-04-01 12:00:00-07:00    189.915445
dtype: float64